In [1]:
import sympy as sp

# Define symbolic variables
s, t, u, a, alpha = sp.symbols('s t u a alpha')
a_last_cycle = sp.Symbol('a_last_cycle')
alpha_last_cycle = 0.002  # from the Maple code
last_cycle = 0.1

# Define expressions
psi0 = sp.exp(-2 * s)
psi_corr = 1 + u / 2 * sp.exp(-1.013 * u)
psi = psi0 * psi_corr * (1 + 0.2119256858 * s * u + a * t**2 - alpha * u**2)

# Derivatives
t1 = sp.diff(psi, s)
t2 = sp.diff(psi, t)
t3 = sp.diff(psi, u)

# i1 expression
i1 = (
    u * (s**2 - t**2) * (t1**2 + t2**2 + t3**2)
    + 2 * t3 * (s * (u**2 - t**2) * t1 + t * (s**2 - u**2) * t2)
    - psi**2 * (4 * 2 * s * u - s**2 + t**2)
)

i1 = sp.expand(i1)
i1 = sp.simplify(i1)

# Evaluate i1 with integrals and s=0
i1_eval = sp.simplify(
    sp.integrate(
        sp.integrate(
            sp.integrate(i1.subs(s, 0), (t, 0, u)), (u, 0, s)
        ), (s, 0, s)
    )
)

# n1 expression and evaluation
n1 = u * (s**2 - t**2) * psi**2
n1_eval = sp.simplify(
    sp.integrate(
        sp.integrate(
            sp.integrate(n1.subs(s, 0), (t, 0, u)), (u, 0, s)
        ), (s, 0, s)
    )
)

# Energy expression
E = sp.simplify(i1_eval / n1_eval)
Ealpha = E.subs(alpha, alpha_last_cycle)
Ea = E.subs(a, a_last_cycle)

# Minimization loop
from sympy.utilities.lambdify import lambdify
import numpy as np
from scipy.optimize import minimize_scalar

# Lambdify expressions for numerical minimization
Ealpha_func = lambdify(a, Ealpha, 'numpy')
Ea_func = lambdify(alpha, Ea, 'numpy')

for k in range(1, 11):
    # Minimize with fixed alpha (vary a)
    res_Ealpha = minimize_scalar(
        Ealpha_func, bounds=(0.5 * a_last_cycle, 3 * a_last_cycle), method='bounded'
    )
    
    # Minimize with fixed a (vary alpha)
    res_Ea = minimize_scalar(
        Ea_func, bounds=(0.5 * alpha_last_cycle, 50 * alpha_last_cycle), method='bounded'
    )

    print(f"min_Ealpha = {res_Ealpha.fun}, a = {res_Ealpha.x}")
    print(f"min_Ea = {res_Ea.fun}, alpha = {res_Ea.x}")

    # Update expressions
    Ealpha = E.subs(a, res_Ea.x)
    Ea = E.subs(alpha, res_Ealpha.x)


KeyboardInterrupt: 

In [3]:
import sympy as sp

# Define symbolic variables
s, t, u, a, alpha = sp.symbols('s t u a alpha')
a_last_cycle = sp.Symbol('a_last_cycle')
alpha_last_cycle = 0.002  # from the Maple code
last_cycle = 0.1

# Define expressions
psi0 = sp.exp(-2 * s)
psi_corr = 1 + u / 2 * sp.exp(-1.013 * u)
psi = psi0 * psi_corr * (1 + 0.2119256858 * s * u + a * t**2 - alpha * u**2)

# Derivatives
t1 = sp.diff(psi, s)
t2 = sp.diff(psi, t)
t3 = sp.diff(psi, u)

# i1 expression
i1 = (
    u * (s**2 - t**2) * (t1**2 + t2**2 + t3**2)
    + 2 * t3 * (s * (u**2 - t**2) * t1 + t * (s**2 - u**2) * t2)
    - psi**2 * (4 * 2 * s * u - s**2 + t**2)
)

In [5]:
n1 = u * (s**2 - t**2) * psi**2

In [6]:
n1

u*(s**2 - t**2)*(u*exp(-1.013*u)/2 + 1)**2*(a*t**2 - alpha*u**2 + 0.2119256858*s*u + 1)**2*exp(-4*s)

In [ ]:
!pip install sympy